In [1]:
import numpy as np

from langevian_protein import calculate_configuration_force

In [4]:
protein_chain = ['H', 'P', 'P', 'H', 'H']
coordinates = np.array([
    [0,0,1],
    [0,0,2],
    [0,0,3],
    [0,0,4],
    [0,0,5]
])

In [5]:
V_0, forces_0 = calculate_configuration_force(coordinates, protein_chain, 20, 1)

In [7]:
forces_0, V_0

(array([[ 0.        ,  0.        ,  0.13350171],
        [ 0.        ,  0.        ,  0.12838964],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        , -0.13203758],
        [ 0.        ,  0.        , -0.12985377]]),
 -0.13315560198848553)